%load_ext autoreload
%autoreload 2
https://blog.csdn.net/m0_46413065/article/details/129917287

In [70]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [71]:
data_g2 = pd.read_csv("./Data/Kaohsiung/g2.csv",delimiter=';')
data_g3 = pd.read_csv("./Data/Kaohsiung/g3.csv",delimiter=';')
data_g4 = pd.read_csv("./Data/Kaohsiung/g4.csv",delimiter=';')
data = [data_g2]

features = ['date_time']
targets = ['E', 'N', 'H']
group_id_i = 2

for i in range(1):
    data[i]['date_time'] = pd.to_datetime(data[i]['date_time'])
    data[i]['date_time'] = data[i]['date_time'].astype(int) / (10 ** 11)
    data[i]['date_time'] = data[i]['date_time'].astype(int)
    data[i] = data[i][features+targets]
    #data[i]['group_id'] = group_id_i
    group_id_i += 1
    
data = pd.concat(data,ignore_index=True)
data['group_id'] = 0

In [72]:
data.head()

,date_time,E,N,H,group_id
0,17055522,224095.7991,2.551570e+06,1518.4140,0
1,17055528,224095.7980,2.551570e+06,1518.4135,0
2,17055744,224095.7973,2.551570e+06,1518.3954,0
3,17055750,224095.8001,2.551570e+06,1518.3838,0
4,17055762,224095.8011,2.551570e+06,1518.3934,0


In [73]:
data.tail()

,date_time,E,N,H,group_id
31179,17355504,224095.7558,2.551570e+06,1518.5164,0
31180,17355510,224095.7558,2.551570e+06,1518.5165,0
31181,17355516,224095.7561,2.551570e+06,1518.5175,0
31182,17355522,224095.7536,2.551570e+06,1518.5217,0
31183,17355600,224095.7455,2.551570e+06,1518.5070,0


In [74]:
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import MultiNormalizer,TorchNormalizer

max_prediction_length = 7#*144
max_encoder_length = 7*144

training = TimeSeriesDataSet(
    data[:len(data)-max_prediction_length],
    group_ids = ["group_id"],
    target = targets,
    time_idx = "date_time",
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_encoder_length=max_encoder_length//2,  # 似乎要自己設置(默認值好像太大?)
    min_prediction_length=1,  #同上
    time_varying_unknown_reals = targets,
    target_normalizer=MultiNormalizer([TorchNormalizer() for _ in targets]),
    allow_missing_timesteps=True
)

validation = TimeSeriesDataSet.from_dataset(training,data,predict=True,stop_randomization=True)

batch_size = 32

train_dataloader = training.to_dataloader(train=True,batch_size=batch_size,num_workers=4)
val_dataloader = validation.to_dataloader(train=False,batch_size=batch_size*10,num_workers=4)

In [75]:
# from pytorch_forecasting import Baseline

# actuals = torch.cat([y[0] for x, (y,weight) in iter(val_dataloader)])
# baseline_predictions = Baseline().predict(val_dataloader)
# # 假設你希望將 `baseline_predictions` 移動到相同的設備
# baseline_predictions = baseline_predictions.to(actuals.device)

# # 現在兩者在相同設備上，可以進行計算
# result = (actuals - baseline_predictions).abs().mean().item()

from pytorch_forecasting import Baseline

# 取得 `actuals`，支援多個 target
actuals = torch.cat([y if isinstance(y, torch.Tensor) else y[0] for x, (y, weight) in iter(val_dataloader)])

# 獲取 baseline 預測值
baseline_predictions = Baseline().predict(val_dataloader)

# 確保 `baseline_predictions` 是 tensor
if isinstance(baseline_predictions, list):
    baseline_predictions = torch.stack(baseline_predictions, dim=1)  # 確保維度對應

# 確保 baseline_predictions 在相同設備
baseline_predictions = baseline_predictions.to(actuals.device)

# 計算 MAE
result = (actuals - baseline_predictions).abs().mean(dim=[0, 2])  # 針對每個 target 計算 MAE
print("MAE for each target:", result.tolist())


/home/undergrad/miniconda3/envs/114/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/undergrad/miniconda3/envs/114/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 

MAE for each target: [0.0, 2327474.25, 222577.234375]


In [76]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TemporalFusionTransformer, QuantileLoss, MultiLoss

early_stop_callback = EarlyStopping(monitor="val_loss",min_delta=1e-4,patience=5,verbose=True,mode="min")
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")

trainer = pl.Trainer(
    max_epochs=15,
    accelerator = 'gpu',
    devices=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    accumulate_grad_batches=4,
    callbacks=[lr_logger,early_stop_callback],
    logger=logger
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.001,
    hidden_size=100,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=40,
    output_size=[1,1,1], #len(targets),
    loss=MultiLoss([QuantileLoss(quantiles=[0.5]),QuantileLoss(quantiles=[0.5]),QuantileLoss(quantiles=[0.5])]),
    log_interval=10,
    reduce_on_plateau_patience=4
)

trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | MultiLoss                       | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 0      | train
3  | prescalers                         | ModuleDict                      | 320    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 12.1 K | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 36.9 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 0      | train
7  | static_co

Epoch 0: 100%|██████████| 1150/1150 [01:21<00:00, 14.19it/s, v_num=9, train_loss_step=0.0144, val_loss=0.0344, train_loss_epoch=0.0263]

Metric val_loss improved. New best score: 0.034


Epoch 1: 100%|██████████| 1150/1150 [01:21<00:00, 14.16it/s, v_num=9, train_loss_step=0.0182, val_loss=0.0171, train_loss_epoch=0.0162] 

Metric val_loss improved by 0.017 >= min_delta = 0.0001. New best score: 0.017


...

Epoch 12: 100%|██████████| 1150/1150 [01:21<00:00, 14.15it/s, v_num=9, train_loss_step=0.0115, val_loss=0.00572, train_loss_epoch=0.0131] 

Metric val_loss improved by 0.001 >= min_delta = 0.0001. New best score: 0.006


Epoch 14: 100%|██████████| 1150/1150 [01:21<00:00, 14.15it/s, v_num=9, train_loss_step=0.00451, val_loss=0.00696, train_loss_epoch=0.0132]

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 1150/1150 [01:21<00:00, 14.10it/s, v_num=9, train_loss_step=0.00451, val_loss=0.00696, train_loss_epoch=0.0132]


In [79]:
best_model_path = trainer.checkpoint_callback.best_model_path
print(best_model_path)
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

lightning_logs/lightning_logs/version_9/checkpoints/epoch=14-step=4320.ckpt


In [ ]:
#actuals = torch.cat([y for x,y in iter(val_dataloader)]).cpu().numpy()
actuals = torch.cat([y if isinstance(y, torch.Tensor) else y[0] for x, (y, weight) in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)

from sklearn.metrics import mean_absolute_error, mean_squared_error 

#mae_T = mean_absolute_error(actuals, predictions)
#rmse_T = np.sqrt(mean_squared_error(actuals, predictions))

#print(mae_T,'\n',rmse_T)

# 对于每个目标，计算 MAE 和 RMSE
for target_idx in range(3):  # 假设 actuals 和 predictions 的形状是 (num_samples, num_targets)
    actual_target = actuals[0,target_idx].cpu().numpy()
    predicted_target = predictions[target_idx]

    mae = mean_absolute_error(actual_target, predicted_target)
    rmse = np.sqrt(mean_squared_error(actual_target, predicted_target))

    print(f"Target {target_idx + 1} MAE: {mae:.4f}")
    print(f"Target {target_idx + 1} RMSE: {rmse:.4f}")

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


TypeError: numpy() takes 0 positional arguments but 1 was given